In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Define the paths
MAIN_FOLDER = '/app'
DATA_FOLDER = os.path.join(MAIN_FOLDER, 'data')

# Paths to original data
TRAIN_IMAGES_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Training_Input')
VAL_IMAGES_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Validation_Input')
TEST_IMAGES_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Test_Input')
RAD_IMAGES_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task3_Test_Images')


# Paths to save data with no hair
TRAIN_IMAGES_PATH_NO_HAIR = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Training_Input_No_Hair')
VAL_IMAGES_PATH_NO_HAIR = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Validation_Input_No_Hair')
TEST_IMAGES_PATH_NO_HAIR = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Test_Input_No_Hair')
RAD_IMAGES_PATH_NO_HAIR = os.path.join(DATA_FOLDER, 'ISIC2018_Task3_Test_Images_No_Hair')


# Function to remove hair
def remove_hair(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Increase contrast using adaptive histogram equalization (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    
    # Use morphological operations to detect hair
    kernel_size = (9, 9)  # Smaller kernel size for more precise hair detection
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    
    # Use a threshold to create a binary mask of the hair
    _, binary_mask = cv2.threshold(blackhat, 15, 255, cv2.THRESH_BINARY)
    
    # Dilate the mask to cover hair more effectively
    dilated_mask = cv2.dilate(binary_mask, np.ones((3, 3), np.uint8), iterations=1)
    
    # Inpaint the original image using the dilated mask
    inpainted_image = cv2.inpaint(image, dilated_mask, inpaintRadius=2, flags=cv2.INPAINT_TELEA)
    
    return inpainted_image

# Function to process and save images without hair
def process_and_save_images(images_path, save_path):
    # Ensure the save path exists
    os.makedirs(save_path, exist_ok=True)
    
    # List all images in the directory
    image_files = os.listdir(images_path)
    
    # Use tqdm to show progress bar
    for image_file in tqdm(image_files, desc=f"Processing {os.path.basename(images_path)}"):
        # Read the image
        image_path = os.path.join(images_path, image_file)
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error: {image_file} could not be read.")
            continue
        
        # Remove hair from the image
        image_no_hair = remove_hair(image)
        
        # Save the processed image
        save_image_path = os.path.join(save_path, image_file)
        cv2.imwrite(save_image_path, image_no_hair)

# Process and save the training, validation, and test datasets with progress bars
process_and_save_images(TRAIN_IMAGES_PATH, TRAIN_IMAGES_PATH_NO_HAIR)
process_and_save_images(VAL_IMAGES_PATH, VAL_IMAGES_PATH_NO_HAIR)
process_and_save_images(TEST_IMAGES_PATH, TEST_IMAGES_PATH_NO_HAIR)
process_and_save_images(RAD_IMAGES_PATH, RAD_IMAGES_PATH_NO_HAIR)

print("All processing completed.")

Processing ISIC2018_Task3_Test_Images: 100%|██████████| 1511/1511 [01:54<00:00, 13.23it/s]

All processing completed.
